In [1]:
import sys
sys.path.append("../input/iterstrat")
import pandas as pd
import re
import string
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from transformers import AutoModel,AutoTokenizer
import torch
import torch.nn.functional as F
from sklearn.linear_model import Ridge
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb
from tqdm import tqdm
import numpy as np
import warnings
from config import config

warnings.filterwarnings("ignore")                                                   

namespace(accumulate_grad_batches=None, bert='microsoft/deberta-v3-large', bs=4, ck='/home/wangjingqi/input/ck/fpell', device_ids=[7], epochs=5, freeze=None, head_lr=0.0001, layer_start=-1, llrd=0.4, llrd_interval=1, log='/home/wangjingqi/fpell-pl/log', max_len=640, model_fname='deberta-v3-large_lr1-wd-0.4-0.001-1-1-1-640-5-2022', nfolds=5, patience=10, precision=16, prefix='lr1-wd', reinit_layers=1, seed=2022, submit='/home/wangjingqi/input/dataset/fpell/sample_submission.csv', test='/home/wangjingqi/input/dataset/fpell/test.csv', train='/home/wangjingqi/input/dataset/fpell/train/train.csv', used_folds=[0, 1, 2, 3, 4], val_check_interval=1.0, wd=0.001)


In [5]:
4900/25

196.0

**Load data**

In [2]:
df = pd.read_csv(config.train)
df["src"] = "train"
ss = pd.read_csv(config.submit)
target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions',]
print(df.shape)
df.head()

(3911, 13)


,Unnamed: 0,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,topic,topic_prob,topic_name,src
0,0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,2,1.000000,students online school classes,train
1,1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,1,1.000000,advice people ask multiple,train
2,2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,10,1.000000,sports average school students,train
3,3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,1,1.000000,advice people ask multiple,train
4,4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,12,0.715767,community service community service help,train


**MultilabelStratifiedKFold**

In [4]:
class fpe_dataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.df = df.reset_index(drop=True)
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        text = self.df.loc[idx,"full_text"]

        return text
class collator():
    def __init__(self,max_len,pretrained_path,) -> None:
        self.max_len = max_len
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_path)
    def __call__(self, data):
            
        X = self.tokenizer(
                data,
                None,max_length=self.max_len,
                padding='max_length',truncation=True,return_tensors="pt")
        return X

def preprocess(text):
    text = text.split(" ")
    while 2>1:
        if text[-1] in [""," ","\n","\r"]:
            text.pop(-1)
        else:
            break
    return " ".join(text)
df['full_text'] = df['full_text'].apply(preprocess)

**Model**

In [5]:

import torch 
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoModel,AutoTokenizer
import pytorch_lightning as pl
from sklearn.metrics import mean_squared_error
import numpy as np
from transformers import get_cosine_schedule_with_warmup,AdamW,get_linear_schedule_with_warmup
from copy import deepcopy
import math


class MaxPooling(nn.Module):
    def __init__(self) -> None:
        super().__init__()
    def forward(self,hidden_state,attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_state.size()).float()
        hidden_state[input_mask_expanded == 0] = -1e9  # Set padding tokens to large negative value
        max_embeddings = torch.max(hidden_state, 1)[0]
        return max_embeddings

class MeanPooling(nn.Module):
    def __init__(self) -> None:
        super().__init__()
    def forward(self,hidden_state, attention_mask):

        input_mask_expanded = (
            attention_mask.unsqueeze(-1).expand(hidden_state.size())
        )
        mean_embeddings = torch.sum(hidden_state * input_mask_expanded, 1) / torch.clamp(
            input_mask_expanded.sum(1), min=1e-9)
        return mean_embeddings

class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start, layer_weights = None):
        super().__init__()
        if layer_start < 0:
            self.layer_start =num_hidden_layers+1+layer_start
        else:
            self.layer_start = layer_start
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - self.layer_start), dtype=torch.float)
            )

    def forward(self, features):

        all_layer_embedding = torch.stack(features)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]

        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average



class deberta(nn.Module):
    def __init__(self,config) -> None:
        super().__init__()
        self.config = config
        self.encoder = AutoModel.from_pretrained(config.bert,attention_probs_dropout_prob=0.,hidden_dropout_prob=0.,output_hidden_states=True)
        hidden_size = self.encoder.config.hidden_size
        num_attention_heads = self.encoder.config.num_attention_heads
        num_hidden_layers =  self.encoder.config.num_hidden_layers
        
        self.WeightedLayerPooling =nn.Sequential(WeightedLayerPooling(num_hidden_layers=num_hidden_layers,layer_start=config.layer_start,))
        self.Pooler = MeanPooling()
        self.head =nn.Sequential(nn.Linear(hidden_size,hidden_size),nn.GELU(),nn.Linear(hidden_size,6))
    def forward(self,input_ids,attention_mask,token_type_ids):

        b,l = input_ids.shape
        x = dict(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        #for deberta
        encoder = self.encoder(**x)
        all_hidden_states =encoder.hidden_states
        hidden_states = self.WeightedLayerPooling(all_hidden_states)
        hidden_states = self.Pooler(hidden_states,attention_mask)#bs,768
        return hidden_states



In [6]:

import os
from torch.utils.data import DataLoader
def get_embeddings(mp='',df=None,verbose=True):

    ck = torch.load(mp,map_location=torch.device('cpu'))
    model_config = ck["config"]
    # pretrained = os.path.join("../input/deberta",os.path.basename(model_config.bert))
    # model_config.bert = os.path.join(pretrained,"model")
    model_config.tokenizer=model_config.bert#os.path.join(pretrained,"tokenizer")
    model=deberta(model_config)
    model.load_state_dict(state_dict=ck["model"]) 
    
    dataset = fpe_dataset(df)
    dataloader = DataLoader(dataset,batch_size =32, shuffle = False, pin_memory=True,collate_fn = collator(max_len=model_config.max_len,pretrained_path=model_config.tokenizer),drop_last=False)
    model.cuda()
    model.eval()
    model.float()

    all_text_feats = []
    for batch in tqdm(dataloader,total=len(dataloader)):
        input_ids = batch["input_ids"].cuda()
        attention_mask = batch["attention_mask"].cuda()
        token_type_ids = batch["token_type_ids"].cuda()
        with torch.no_grad():
            model_output = model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        # Normalize the embeddings
        sentence_embeddings = F.normalize(model_output, p=2, dim=1)
        sentence_embeddings =  sentence_embeddings.detach().cpu().numpy()
        all_text_feats.extend(sentence_embeddings)
    all_text_feats = np.array(all_text_feats)
    if verbose:
        print('Train embeddings shape',all_text_feats.shape)
        
    return all_text_feats

In [7]:
all_feats = []

In [8]:
def get_path(m_dirs):
    model_list = []
    for m in m_dirs:
        model_list +=[os.path.join(m,i) for i in os.listdir(m)]
    return model_list

m_dirs = ["deberta-v3-base_lr1-wd-0.4-0-1-1-1-640-5-2022",
"deberta-v3-base_lr1-wd-0.4-0-1-1-1-1024-5-2022",]
m_dirs = [os.path.join("",i) for i in m_dirs]
model_path =get_path(m_dirs)
print(len(model_path),model_path)
for m in model_path:
    text_feats = get_embeddings(m,df)
    all_feats.append(text_feats)

10 ['/home/wangjingqi/input/ck/fpell/deberta-v3-base_lr1-wd-0.4-0-1-1-1-640-5-2022/deberta-v3-base_lr1-wd-0.4-0-1-1-1-640-5-2022_2.ckpt', '/home/wangjingqi/input/ck/fpell/deberta-v3-base_lr1-wd-0.4-0-1-1-1-640-5-2022/deberta-v3-base_lr1-wd-0.4-0-1-1-1-640-5-2022_4.ckpt', '/home/wangjingqi/input/ck/fpell/deberta-v3-base_lr1-wd-0.4-0-1-1-1-640-5-2022/deberta-v3-base_lr1-wd-0.4-0-1-1-1-640-5-2022_0.ckpt', '/home/wangjingqi/input/ck/fpell/deberta-v3-base_lr1-wd-0.4-0-1-1-1-640-5-2022/deberta-v3-base_lr1-wd-0.4-0-1-1-1-640-5-2022_1.ckpt', '/home/wangjingqi/input/ck/fpell/deberta-v3-base_lr1-wd-0.4-0-1-1-1-640-5-2022/deberta-v3-base_lr1-wd-0.4-0-1-1-1-640-5-2022_3.ckpt', '/home/wangjingqi/input/ck/fpell/deberta-v3-base_lr1-wd-0.4-0-1-1-1-1024-5-2022/deberta-v3-base_lr1-wd-0.4-0-1-1-1-1024-5-2022_0.ckpt', '/home/wangjingqi/input/ck/fpell/deberta-v3-base_lr1-wd-0.4-0-1-1-1-1024-5-2022/deberta-v3-base_lr1-wd-0.4-0-1-1-1-1024-5-2022_4.ckpt', '/home/wangjingqi/input/ck/fpell/deberta-v3-base_lr1-w

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have b

Train embeddings shape (3911, 768)


Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have b

Train embeddings shape (3911, 768)


Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have b

Train embeddings shape (3911, 768)


Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have b

In [ ]:
all_feats = np.concatenate(all_feats,axis=1)

In [ ]:
print('Our concatenated embeddings have shape', all_feats.shape )

In [ ]:
np.save("/home/wangjingqi/input/ck/fpell/all_feats.npy",all_feats)